In [1]:
# 第二类负采样数据--不相关问题的未被接受答案

In [2]:
import pandas as pd
from random import sample
qa_set_p =  "D:/OneDriveEdu/file/project2/QAData/GetDataSet3/qa_set_u21.csv"
sim_q_p =  "D:/OneDriveEdu/file/project2/QAData/GetDataSet3/sim_q_u21.csv"
negSamples_p = "D:/OneDriveEdu/file/project2/QAData/GetDataSet3/neg2Samples_u21.csv"
negSamples1000_p = "D:/OneDriveEdu/file/project2/QAData/GetDataSet3/neg2Samples_u21.1000.csv"
# 一定要去重复项目！
qa_set = pd.read_csv(qa_set_p).drop(columns='Unnamed: 0').drop(columns='Unnamed: 0.1').drop_duplicates(['a_text', 'q_text'], keep='last')
# qa_set.columns=['q_id','q_text', 'u_id', 'a_text', 'r', 'q_time', 'a_time']
qa_set

,q_id,q_time,q_text,u_id,a_id,a_text,a_time,r
0,53625567,1543986891,in java how many constructor can we create in ...,4725,53835539,tl;dr for a class with reasonable functionalit...,1545144381,"[0.0, 1.0, 0.0, 0.0]"
1,52690965,1538932562,randomaccessfile unexpected behavior for seek ...,4725,52691194,in the former the offset is into the file you ...,1538934276,"[0.0, 1.0, 0.0, 0.0]"
2,52693305,1538949775,java .split() out of boundsi have a problem wi...,4725,52693790,"so you have examples like this #ch id="""" tvg-n...",1538954935,"[0.0, 1.0, 0.0, 0.0]"
3,53976576,1546163020,generic method with stackmy code: public stati...,4725,53977248,if you really wanted to use this algorithm for...,1546169556,"[1.0, 0.0, 0.0, 0.0]"
4,53958810,1546000933,two generic interfaces referencing each otheri...,4725,53959024,public abstract class tileimpl&lt;s&gt; implem...,1546002108,"[1.0, 0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...
33180,53599514,1543860856,getting specified data from model as json resp...,10668441,53603164,assuming you're using jackson you can add cust...,1543877569,"[1.0, 0.0, 0.0, 0.0]"
33181,53552715,1543561038,how to read data from csv and write it in html...,10668441,53555494,your modelmap will only have the last row from...,1543573090,"[1.0, 0.0, 0.0, 0.0]"
33182,53428378,1542880980,why im getting this error no mapping found for...,10668441,53430947,your view resolver doesn't seem to have a view...,1542889379,"[0.0, 1.0, 0.0, 0.0]"
33183,53382369,1542660250,having a `java.lang.stackoverflowerror: null` ...,10668441,53384239,i'd start by adding the following to you appli...,1542670986,"[1.0, 0.0, 0.0, 0.0]"


In [3]:
sim_q = pd.read_csv(sim_q_p).drop(columns='Unnamed: 0')
# sim_q = sim_q.head(10)
q_id_list = list(sim_q['q_id'])
sim_q

,q_id,sim_q
0,48603980,"[48603980, 53351111, 53171685, 50947995, 53775..."
1,50498949,"[50498949, 49176390, 51579497, 53597053, 50967..."
2,52627023,"[52627023, 52244285, 50169972, 50259051, 52190..."
3,51713421,"[51713421, 48737232, 52096443, 49861669, 49894..."
4,50714411,"[50714411, 50921837, 48259012, 52373766, 51928..."
...,...,...
26124,48048396,"[48048396, 49509625, 50428037, 51188982, 52302..."
26125,48048167,"[48048167, 50382843, 51608509, 49045052, 53380..."
26126,48047270,"[48047270, 49262755, 49597427, 53585274, 49673..."
26127,48047192,"[48047192, 50875114, 53476055, 52310031, 53844..."


In [4]:
rand_q_id = sample(q_id_list, 1)
rand_q_id

[48379995]

In [5]:
from tqdm import tqdm
negSamples = pd.DataFrame()
for index, row in tqdm(list(sim_q.iterrows())):
    q_id = row['q_id']
    sim_q_list = eval(row['sim_q'])
    # print(len(sim_q_list))

    if q_id not in list(qa_set["q_id"]):
        continue

    p = pd.DataFrame()
    neg_item = pd.DataFrame()
    while True:
        rand_q_id = sample(q_id_list, 1)[0]
        if rand_q_id in sim_q_list:
            # print(rand_q_id)
            continue
        q_df = qa_set[qa_set['q_id']==rand_q_id]
        p = pd.concat([p, q_df[q_df['r']=='[0.0, 1.0, 0.0, 0.0]']])
        
        if len(p) > 0:
            break
    neg_item['q_id'] = [row['q_id']]
    neg_item['u_id'] = [list(p['u_id'])[0]]
    q_text = qa_set[qa_set['q_id']==row['q_id']][0:1]
    neg_item['q_text'] = [list(q_text['q_text'])[0]]
    neg_item['a_text'] = [list(p['a_text'])[0]]
    neg_item['q_time'] = [list(p['q_time'])[0]]
    neg_item['a_time'] = [list(p['a_time'])[0]]
    neg_item['r'] = [[0.0,0.0,0.0,1.0]]
    negSamples = pd.concat([negSamples, neg_item])
negSamples[negSamples.isnull()==True].isnull().sum()

100%|██████████| 26129/26129 [09:27<00:00, 46.05it/s]


q_id      26129
u_id      26129
q_text    26129
a_text    26129
q_time    26129
a_time    26129
r         26129
dtype: int64

In [6]:
negSamples

,q_id,u_id,q_text,a_text,q_time,a_time,r
0,48603980,9763253,java - write call to socket output stream bloc...,solution: write like this: void placement(rela...,1535992900,1535993675,"[0.0, 0.0, 0.0, 1.0]"
0,50498949,1753073,how to convert utc date string and remove the ...,in onloadfinished method of mainactivity class...,1536467500,1536473438,"[0.0, 0.0, 0.0, 1.0]"
0,52627023,1553851,how can i create profiles with spring boot + ....,use arrays.stream(): // end index is exclusive...,1524169452,1524171592,"[0.0, 0.0, 0.0, 1.0]"
0,51713421,9164141,pass bigdecimal numbers in javai want to pass ...,you can also disable the warnings if you need ...,1532597159,1562615401,"[0.0, 0.0, 0.0, 1.0]"
0,50714411,5862071,redirection of a jsp page back to itself with ...,you can do so with: person.stream() .filter(x ...,1522953212,1522953763,"[0.0, 0.0, 0.0, 1.0]"
...,...,...,...,...,...,...,...
0,48048396,1262248,receiver to alarm activity bundle returning nu...,you can simply take the company id from the ob...,1532278327,1532278670,"[0.0, 0.0, 0.0, 1.0]"
0,48048167,9959152,unable to resolve generic error in rest apii a...,from the documentation public socket(string ho...,1534019590,1534019856,"[0.0, 0.0, 0.0, 1.0]"
0,48047270,1383240,how to insert an object into a linkedlist depe...,if the servers are just a replication of each ...,1539286445,1539286857,"[0.0, 0.0, 0.0, 1.0]"
0,48047192,5772882,apache poi read txt file as an excel filecould...,"locale de = locale.forlanguagetag(""de""); weekf...",1526536640,1526540066,"[0.0, 0.0, 0.0, 1.0]"


In [7]:
negSamples.to_csv(negSamples_p)
negSamples.head(1000).to_csv(negSamples1000_p)